# Hands-On 9

Implement a hash table and upload your code to github:

Use the multiplication AND division method for your hash function
Note your code should be generic enough to allow for ANY hash function
For simplicity assume your keys are integers and the values (data) are integers
Use collision resolution by chaining
Use a doubly linked list and you must write your own (so for example you can't use "list" in C++)
You are only allowed to use C-style array's for this implementation (so for example no C++ vectors)
Your Hash table should grow and shrink
When it's full double the array size and re-hash everything
When it's becoming empty e.g. 1/4 empty, then half the size of the array and re-hash everything

In [3]:
class DoublyLinkedList:
    class Node:
        def __init__(self, key, value):
            self.key = key
            self.valueHead = DoublyLinkedList.ValueNode(value)
            self.next = None
            self.prev = None

    class ValueNode:
        def __init__(self, value):
            self.value = value
            self.next = None

    def __init__(self):
        self.head = None
        self.tail = None

    def insert(self, key, value):
        node = self.find_node(key)
        if node:
            # If the key already exists, append the new value to the value list
            new_value_node = DoublyLinkedList.ValueNode(value)
            last_value_node = node.valueHead
            while last_value_node.next:
                last_value_node = last_value_node.next
            last_value_node.next = new_value_node
        else:
            # Create a new node if the key doesn't exist
            new_node = DoublyLinkedList.Node(key, value)
            if self.head is None:
                self.head = self.tail = new_node
            else:
                self.tail.next = new_node
                new_node.prev = self.tail
                self.tail = new_node

    def delete(self, key):
        node = self.find_node(key)
        if node:
            if node.prev:
                node.prev.next = node.next
            if node.next:
                node.next.prev = node.prev
            if node == self.head:
                self.head = node.next
            if node == self.tail:
                self.tail = node.prev

    def find(self, key):
        node = self.find_node(key)
        if node:
            return node.valueHead.value
        return None

    def find_node(self, key):
        current = self.head
        while current:
            if current.key == key:
                return current
            current = current.next
        return None

    def get_all_values(self, key):
        node = self.find_node(key)
        if node:
            values = []
            value_node = node.valueHead
            while value_node:
                values.append(value_node.value)
                value_node = value_node.next
            return values
        return []

class HashTable:
    LOAD_FACTOR = 0.75
    SHRINK_FACTOR = 0.25
    INITIAL_CAPACITY = 16

    def __init__(self, hash_function):
        self.capacity = self.INITIAL_CAPACITY
        self.table = [DoublyLinkedList() for _ in range(self.capacity)]
        self.size = 0
        self.hash_function = hash_function

    def insert(self, key, value):
        if (self.size / self.capacity) >= self.LOAD_FACTOR:
            self.resize(self.capacity * 2)
        index = self.hash_function.hash(key, self.capacity)
        self.table[index].insert(key, value)
        self.size += 1
        self.display()

    def remove(self, key):
        index = self.hash_function.hash(key, self.capacity)
        self.table[index].delete(key)
        self.size -= 1
        if (self.size / self.capacity) <= self.SHRINK_FACTOR and self.capacity > self.INITIAL_CAPACITY:
            self.resize(self.capacity // 2)

    def resize(self, new_capacity):
        old_table = self.table
        self.capacity = new_capacity
        self.table = [DoublyLinkedList() for _ in range(self.capacity)]
        self.size = 0
        for linked_list in old_table:
            node = linked_list.head
            while node:
                value_node = node.valueHead
                while value_node:
                    self.insert(node.key, value_node.value)
                    value_node = value_node.next
                node = node.next

    def display(self):
        print("Hash Table (Chaining Display):")
        print("-------------------------------")
        for linked_list in self.table:
            node = linked_list.head
            while node:
                print(f"{node.key}: {linked_list.get_all_values(node.key)}")
                node = node.next
        print("-------------------------------")

    def find(self, key):
        index = self.hash_function.hash(key, self.capacity)
        return self.table[index].find(key)

    class HashFunction:
        def hash(self, key, capacity):
            raise NotImplementedError

    class DivisionHashFunction(HashFunction):
        def hash(self, key, capacity):
            return key % capacity

    class MultiplicationHashFunction(HashFunction):
        A = 0.6180339887  # (1 - sqrt(5)) / 2

        def hash(self, key, capacity):
            return int(capacity * ((key * self.A) % 1))

# Example Usage
if __name__ == "__main__":
    # Use Division Method
    hash_table = HashTable(HashTable.DivisionHashFunction())
    hash_table.insert(1, 100)
    hash_table.insert(1, 150)
    hash_table.insert(2, 200)
    hash_table.insert(2, 250)
    hash_table.insert(3, 300)
    hash_table.insert(4, 400)

    hash_table.display()


Hash Table (Chaining Display):
-------------------------------
1: [100]
-------------------------------
Hash Table (Chaining Display):
-------------------------------
1: [100, 150]
-------------------------------
Hash Table (Chaining Display):
-------------------------------
1: [100, 150]
2: [200]
-------------------------------
Hash Table (Chaining Display):
-------------------------------
1: [100, 150]
2: [200, 250]
-------------------------------
Hash Table (Chaining Display):
-------------------------------
1: [100, 150]
2: [200, 250]
3: [300]
-------------------------------
Hash Table (Chaining Display):
-------------------------------
1: [100, 150]
2: [200, 250]
3: [300]
4: [400]
-------------------------------
Hash Table (Chaining Display):
-------------------------------
1: [100, 150]
2: [200, 250]
3: [300]
4: [400]
-------------------------------
